# Project 4b - Molecular Simulation: N-bodies and emergent properties

## Due date TBD

With working code for an $n$-molecule simulation in place (along with periodic boundary conditions) we can start to examine some interesting cases.  In particular, we will perform simulations corresponding to materials in the gas, liquid, and solid phases.  

In [ ]:
import LJParticles as ljp
import numpy as np
import matplotlib.pyplot as plt

### Random placement - gas
One of the trickiest aspects of molecular dynamics simulations is the specification of initial conditions.  This is a result of the strong repulsive forces of the Lennard-Jones force.  If particles are randomly placed too close together, we'll get a rather extreme force that will quickly turn into an unreasonably fast particle and the simulation will explode.  As such, **begin by devising a subroutine that randomly places $N$ particles such that no two are closer than $2^\frac{1}{6} \sigma$.**  Look at Gould 8.4 for inspiration.  Using this routine, randomly place 10 particles in a domain that is size $L\times L$, with $\sigma=\epsilon=1$.  Set initial velocities to normally-distributed random values with standard deviation $\sigma=5$.  Use a time step of $\Delta t=0.001$ and run the simulation for $10$ units of time.  

In [ ]:
# N particles in an LxL box
N = 20
box_size = 10
x0 = np.random.rand(N*2)*box_size
# init velocites to normally distributed random numbers with sd 5
v0 = np.random.randn(N*2)*5
sigma = 1
epsilon = 1
dt = 0.001
t_end = 10
lj_sim = ljp.LJParticleSim(N, box_size, sigma, epsilon, dt, t_end, x0, v0, isRandom=True)

In [ ]:
lj_sim.anim_pygame_interactive()

Such dynamics correspond to a very diffuse gas (note the variable density in different portions of the domain).  Try increasing the density, i.e. by increasing the number of particles for the same size domain.  **How many particles can you add in this way before the randomized method stops working?**.  Why doesn't it work anymore?  

I was able to get 2 to 2.5 times the box size edge (sometimes more, but this was the consistent amount) for the number of particles before the simulation stopped working.  The reason it stops working is that the particles are too close together and the force between them is too large.  This causes the particles to move too quickly and the simulation to explode. Floating point math errors could be another reason for the simulation to stop working.

**Finally, at the maximum plausible number of particles, compute the temperature of the system through time, which is simply the average kinetic energy of all the particles (technically, proportional to, with the Boltzmann constant as the constant of proportionality, but this is really just a matter of changing units).  Does the kinetic energy change through time?  Does it find an equilibrium?**

In [ ]:
kinetic_e, potential_e = lj_sim.calculate_total_energy()
# plot the total energy through time
plt.plot(lj_sim.times, kinetic_e)

### Rectangular lattice - the liquid phase
The situation above essentially corresponds to a vacuum: the pressure is close to zero because the particle are not squeezed together except during occasional repulsive interactions (**The pressure is proportional to the potential energy.  Add a routine to your LennardJones class that computes the Lennard-Jones potential, Gould Eq. 8.2**).  Non-interactive particles such as Argon thus tend to be gases at low pressures.  However, as we squeeze these particles together we can get them to behave like a liquid, which has a uniform density but where particles are still free to move around.  To simulate particles at higher pressure, we'll need to place them so that the density is greater than that achievable through random placment, but also such that the intermolecular forces approximately balance so that we don't get an explosion.  One reasonable starting configuration is a simple cartesian grid.  **Perform the same simulation as above, but with the particles initially placed on a grid with unit spacing.**  Accounting for periodic boundary conditions and again using $L=10$, this implies that you should have 100 particles.  

In [ ]:
# 100 particles on a 10x10 grid with unit spacing i.e. p0 = (x0=0,y0=0), p1 = (x1=0,y1=1), p2 = (x2=0,y2=2), p3 = (x3=0,y3=3), p4 = (x4=0,y4=4), p5 = (x5=0,y5=5), p6 = (x6=0,y6=6), p7 = (x7=0,y7=7), p8 = (x8=0,y8=8), p9 = (x9=0,y9=9), p10 = (x10=1, y10=0), ... , p99 = (x99=9,y99=9)
# the x0 array is arranged like this: [x0, y0, x1, y1, x2, y2, ... x99, y99]
x0 = np.array([0,0,0,1,0,2,0,3,0,4,0,5,0,6,0,7,0,8,0,9,1,0,1,1,1,2,1,3,1,4,1,5,1,6,1,7,1,8,1,9,2,0,2,1,2,2,2,3,2,4,2,5,2,6,2,7,2,8,2,9,3,0,3,1,3,2,3,3,3,4,3,5,3,6,3,7,3,8,3,9,4,0,4,1,4,2,4,3,4,4,4,5,4,6,4,7,4,8,4,9,5,0,5,1,5,2,5,3,5,4,5,5,5,6,5,7,5,8,5,9,6,0,6,1,6,2,6,3,6,4,6,5,6,6,6,7,6,8,6,9,7,0,7,1,7,2,7,3,7,4,7,5,7,6,7,7,7,8,7,9,8,0,8,1,8,2,8,3,8,4,8,5,8,6,8,7,8,8,8,9,9,0,9,1,9,2,9,3,9,4,9,5,9,6,9,7,9,8,9,9])
v0 = np.zeros(200)
N = 100
box_size = 10
sigma = 1
epsilon = 1
dt = 0.001
t_end = 5
lj_sim = ljp.LJParticleSim(N, box_size, sigma, epsilon, dt, t_end, x0, v0)

In [ ]:
x0

In [ ]:
kinetic_e, potential_e = lj_sim.calculate_total_energy()
# plot the total energy through time
plt.plot(lj_sim.times, kinetic_e)
plt.plot(lj_sim.times, potential_e)
plt.legend(['kinetic energy', 'potential energy'])

In [ ]:
lj_sim.anim_pygame_interactive()

**Does the simulation that you produced correspond to the qualitative description of a liquid given in the previous paragraph?**.

It appears to.  The particles are nearly suspended in space and are not moving too quickly.  The density is also uniform throughout the box.  The sim breaks because of floating point errors, but it appears to be a liquid.

### Energy loss - the solid phase
The system of particles here is approximately conservative, and so the temperature of the material (or rather the sum of the kinetic energy and potential energy) should remain constant (you can verify this fact if you wish): there is nowhere for that energy to go.  However, in real systems, heat is transferred from warm things to cool things.  In a macroscopic sense, this process is what you modelled with the coffee cup problem in Homework 1.  We can also do it in a microscopic sense here by including a non-conservative drag force of the form 
$$\mathbf{F}_d = -c_d \mathbf{v}.$$
Such a force is phenomenological, and not grounded in any physical principle, but roughly corresponds to the notion that kinetic energy should be lost to the surrounding medium (which we take to be at absolute zero) proportional to velocity.  **Modify your problem class to include such a drag force.  First, repeat the simulation above, but with a drag coefficient somewhere between 1 and 10.  Describe the configuration of the particles at the end of this simulation ([This](https://en.wikipedia.org/wiki/Close-packing_of_equal_spheres)) might be some helpful context).  Second, plot the temperature as a function of time and comment on how the shape of the curve compares to that of the cooling coffee cup from the homework.**




In [ ]:
# 100 particles on a 10x10 grid with unit spacing i.e. p0 = (x0=0,y0=0), p1 = (x1=0,y1=1), p2 = (x2=0,y2=2), p3 = (x3=0,y3=3), p4 = (x4=0,y4=4), p5 = (x5=0,y5=5), p6 = (x6=0,y6=6), p7 = (x7=0,y7=7), p8 = (x8=0,y8=8), p9 = (x9=0,y9=9), p10 = (x10=1, y10=0), ... , p99 = (x99=9,y99=9)
# the x0 array is arranged like this: [x0, y0, x1, y1, x2, y2, ... x99, y99]
x0 = np.array([0,0,0,1,0,2,0,3,0,4,0,5,0,6,0,7,0,8,0,9,1,0,1,1,1,2,1,3,1,4,1,5,1,6,1,7,1,8,1,9,2,0,2,1,2,2,2,3,2,4,2,5,2,6,2,7,2,8,2,9,3,0,3,1,3,2,3,3,3,4,3,5,3,6,3,7,3,8,3,9,4,0,4,1,4,2,4,3,4,4,4,5,4,6,4,7,4,8,4,9,5,0,5,1,5,2,5,3,5,4,5,5,5,6,5,7,5,8,5,9,6,0,6,1,6,2,6,3,6,4,6,5,6,6,6,7,6,8,6,9,7,0,7,1,7,2,7,3,7,4,7,5,7,6,7,7,7,8,7,9,8,0,8,1,8,2,8,3,8,4,8,5,8,6,8,7,8,8,8,9,9,0,9,1,9,2,9,3,9,4,9,5,9,6,9,7,9,8,9,9])
v0 = np.zeros(200)
N = 100
box_size = 10
sigma = 1
epsilon = 1
dt = 0.001
t_end = 10
lj_sim = ljp.LJParticleSim(N, box_size, sigma, epsilon, dt, t_end, x0, v0, has_drag = True, drag_coefficient = 10e-2)

In [ ]:
lj_sim.anim_pygame_interactive()

At the end of the simulation, the particles are still in a lattice structure, but they are not moving.

In [ ]:
kinetic_e, potential_e = lj_sim.calculate_total_energy()
total_temp = kinetic_e + potential_e

In [ ]:
# plot the total temp through time
plt.plot(lj_sim.times, total_temp)
plt.legend(['total temperature'])